Read COVID genome :

In [32]:
with open('datasets/covid.txt', 'r') as file:
    data = file.read().replace(' ', '').replace('\n', '').rstrip()
    print(data)

...


Implement the BWT transform :

In [33]:
def bwt(sequence):
	sequence += '#'
	table = [sequence[index:] + sequence[:index] for index, _ in enumerate(sequence)]
	table.sort()
	bwt = [rotation[-1] for rotation in table]
	bwt = ''.join(bwt)
	return bwt

def inverse_bwt(sequence):
	table = [col for col in sequence]
	for i in range(len(sequence) - 1):
		table.sort()
		table = [sequence[i] + table[i] for i in range(len(sequence))]
	return table[[row[-1] for row in table].index('#')] 

# test it on an example
original = "hello this is a sequence"
transformed = bwt(original)
print('Burrows-Wheeler Transform: ' + str(transformed))
inverse = inverse_bwt(transformed)
print('Inverse Burrows-Wheeler Transform: ' + str(inverse))

Burrows-Wheeler Transform: ssaoe nchus#t heleleii  q
Inverse Burrows-Wheeler Transform: hello this is a sequence#


And move to front transform :

In [34]:
from string import ascii_lowercase

SYMBOLTABLE = list(ascii_lowercase)
SYMBOLTABLE.append(' ')
SYMBOLTABLE.append('#')
 
def mtft(text, symboltable=SYMBOLTABLE):
    sequence, pad = [], symboltable[::]
    for char in text:
        indx = pad.index(char)
        sequence.append(indx)
        pad = [pad.pop(indx)] + pad
    return ''.join([symboltable[s] for s in sequence])
 
def inverse_mtft(sequence, symboltable=SYMBOLTABLE):
    chars, pad = [], symboltable[::]
    for indx in sequence:
        indx = symboltable.index(indx)
        char = pad[indx]
        chars.append(char)
        pad = [pad.pop(indx)] + pad
    return ''.join(chars)

# test it on an example
original = "hello this is a sequence"
transformed = mtft(original)
print('Move to Front Transform: ' + str(transformed))
inverse = inverse_mtft(transformed)
print('Inverse Move to Front Transform: ' + str(inverse))

Move to Front Transform: hflao ufmuecccibcitvctnc
Inverse Move to Front Transform: hello this is a sequence


And Run-length Encoding :

In [35]:
from re import sub

def rle(text):
    return sub(r'(.)\1*', lambda m: str(len(m.group(0))) + m.group(1),text)
 
def inverse_rle(text):
    return sub(r'(\d+)(\D)', lambda m: m.group(2) * int(m.group(1)),text)

# test it on an example
original = "hello this is a sequence"
transformed = rle(original)
print('Run-length Encoding: ' + str(transformed))
inverse = inverse_rle(transformed)
print('Inverse Run-length Encoding: ' + str(inverse))

Run-length Encoding: 1h1e2l1o1 1t1h1i1s1 1i1s1 1a1 1s1e1q1u1e1n1c1e
Inverse Run-length Encoding: hello this is a sequence


Try to combine these transforms :

In [40]:
def ratio(text):
    comp = rle(mtft(bwt(original)))
    return len(text) / len(comp)

# test it on an example
original = "hello this is a sequence"
transformed = rle(mtft(bwt(original)))
print('Encoded: ' + str(transformed))
inverse = inverse_bwt(inverse_mtft(inverse_rle(transformed)))
print('Decoded: ' + str(inverse))
print('Ratio: ' + "{0:.0f}%".format(ratio(original) * 100))

# test it on DNA sequence
transformed = rle(mtft(bwt(data)))
print('Encoded: ' + str(transformed))
print('Ratio: ' + "{0:.0f}%".format(ratio(data) * 100))



Encoded: 1s1a1b1p1g1 1q1h1l1v1i1#1w1h1f1i1s3b1q1a1e1a1v
Decoded: hello this is a sequence#
Ratio: 52%
Encoded: ...
Ratio: 65007%


We achieve 65000% of compression ratio for COVID genome DNA sequence compression.